In [ ]:
import numpy as np
import keras
import glob
import tensorflow as tf
import pandas as pd
import seaborn as sn

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.applications.resnet import preprocess_input
from keras.applications import ResNet50
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

## Data preparation for ResNet50 model

In [ ]:
classTrain = glob.glob("./data_distribution_for_SVM/train/*")
classTest = glob.glob("./data_distribution_for_SVM/test/*")

In [ ]:
x_train = []
y_train = []
count = 0
classCount = 0
for i in classTrain:
    thing = glob.glob("{}/*".format(i))
    for x in thing:
        image = load_img(x,target_size=(224,224))
        image = img_to_array(image,dtype='int32')
        x_train.append(image)
        y_train.append(classCount)
    classCount+=1

In [ ]:
x_test = []
y_test = []
count = 0
classCount = 0
for i in classTest:
    thing = glob.glob("{}/*".format(i))
    for x in thing:
        image = load_img(x,target_size=(224,224))
        image = img_to_array(image)
        x_test.append(image)
        y_test.append(classCount)
    classCount+=1

In [ ]:
x_trainRes = np.array(x_train)
x_trainRes = preprocess_input(x_trainRes)
y_trainRes = np.array(y_train)

In [ ]:
x_testRes = np.array(x_test)
x_testRes = preprocess_input(x_testRes)
y_testRes = np.array(y_test)

In [ ]:
x_trainRes,x_valRes,y_trainRes,y_valRes = train_test_split(x_trainRes,y_trainRes,test_size=0.2)

In [ ]:
y_trainRes = keras.utils.to_categorical(y_trainRes, 38)
y_valRes = keras.utils.to_categorical(y_trainRes, 38)
y_testRes = keras.utils.to_categorical(y_testRes, 38)

In [ ]:
base_modelRes=ResNet50(include_top=False,input_shape=(224,224,3),weights='imagenet')
base_modelRes.trainable=False

classifierRes=keras.models.Sequential()
classifierRes.add(base_modelRes)
classifierRes.add(Flatten())
classifierRes.add(Dense(38,activation='softmax'))
base_modelRes.summary()
classifierRes.summary()

In [ ]:
classifierRes.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
historyRes = classifierRes.fit(x_trainRes, y_trainRes,epochs=10,validation_data=(x_testRes, y_testRes))

In [ ]:
classifierRes.save('./ResNet50')

In [ ]:
lClassifierRes = keras.models.load_model('C:/Users/frank/Documents/HomeWork/Deep Learning/Plant Disease Project/ResNet50/')

In [ ]:
#Save history for later plotting
hist_dfRes = pd.DataFrame(historyRes.history)
hist_csv_fileRes = 'historyRes.csv'
with open(hist_csv_fileRes, mode='w') as f:
    hist_dfRes.to_csv(f)

In [ ]:
#Graph Model's Accuracy
plt.plot(historyRes.history['accuracy'])
plt.plot(historyRes.history['val_accuracy'])
plt.title('ResNet50 accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('./Figures/ResNet50Accuracy.png')
plt.show()

In [ ]:
#Graph Model's Accuracy
plt.plot(historyRes.history['loss'])
plt.plot(historyRes.history['val_loss'])
plt.title('ResNet50 loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('C:/Users/frank/Documents/AI Project/Figures/DenseClassifierAccuracy.png')
plt.show()

In [ ]:
pred = classifierRes.predict(x_testRes)

In [ ]:
pred = np.argmax(pred,axis=1)
pred =pred.tolist()

In [ ]:
#Confusion Matrix Test
matrix = confusion_matrix(y_test,pred)
matshow(matrix)

In [ ]:
labels = [x for x in range(38)]

In [ ]:
df_cm = pd.DataFrame(matrix, index = labels,
                  columns = labels)
plt.figure(figsize = (25,25))
file = sn.heatmap(df_cm, annot=True)
file = file.get_figure()
file.savefig('./Figures/ResNet50ConfusionMatrix2.png')

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
labelDict = {0:'Apple Scab', 1:'Apple Black Rot', 2:'Cedar Apple Rust', 3:'Apple',4:'Blueberry',
    5:'Cherry',6:'Cherry Powdery Mildew',7:'Corn Cercospora Leaf Spot',8:'Corn Common Rust',
    9:'Corn',10:'Corn Northern Leaf Rot', 11:'Grape Black Rot', 12:'Grape Esca', 13:'Grape',
    14:'Grape Leaf Blight',15:'Orange Haunglongbing',16:'Peach Bacterial Spot',17:'Peach',18:'Bell Pepper',
    19:'Bell Pepper', 20:'Potato Early Blight', 21:'Potato', 22:'Potato Late Blight', 23:'Raspberry', 
    24:'Soybean', 25:'Squash Powdery Mildew',26:'Strawberry',27:'Strawberry Leaf Scorch',
    28:'Tomato Bacterial Spot',29:'Tomato Early Blight',30:'Tomato', 31:'Tomato Late Blight',
    32:'Tomato Leaf Mold', 33:'Tomato Septoria Leaf Spot', 34:'Tomato Spider Mites', 35:'Tomato Target Spot',
    36:'Tomato Mosiac Virus',37:'Tomato Yellow Leaf Curl Virus'}

In [ ]:
def predClassRes(im, labels, model):
    pred = model.predict(im)
    pred = np.argmax(pred,axis=1)
    pred =pred.tolist()
    wrong = 0
    total = len(im)
    
    for i in range(len(pred)):      
        print('Prediction ',i,':')
        print('Predicted class: ',labelDict.get(pred[i]))
        print('Actual class: ', labelDict.get(labels[i]))
        
        fig,ax = plt.subplots(1)
        ax.imshow(x_test[i])
        
        if pred[i] != labels[i]:
            wrong+=1
    print("Total wrong: ", wrong)
    print('Total accuracy: ', ((total-wrong)/total)*100,'%')

In [ ]:
predClassRes(x_testRes[:5],y_test[:5], classifierRes)

## Data preparation for VGG16

In [ ]:
traindir = "./data_distribution_for_SVM/train"
testdir = "./data_distribution_for_SVM/test"
trainDirNames = "./raw/color"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)


batch_size = 128

training_set = train_datagen.flow_from_directory(traindir,
                                                 target_size=(224, 224),
                                                 color_mode="rgb",
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

valid_set = test_datagen.flow_from_directory(testdir,
                                            target_size=(224, 224),
                                            color_mode="rgb",
                                            batch_size=128,
                                            class_mode='categorical')

test_set = test_datagen.flow_from_directory(testdir,
                                            target_size=(224, 224),
                                            color_mode="rgb",
                                            batch_size=199,
                                            class_mode=None,
                                            shuffle=False)

In [ ]:
#Example of image after reshaping
fig,ax = plt.subplots(1)
thing = training_set[55]
thing = thing[0]
thing = thing[127]
ax.imshow(thing)
plt.imsave('./Figures/ImageExample.png',thing)

In [ ]:
#Dataset example distributions
lb = training_set.labels.astype(np.int)
unique, counts = np.unique(lb, return_counts=True)
plt.bar(unique, counts)
plt.xticks(np.arange(38, step = 5))
plt.title('Train Data Class Distribution')
plt.ylabel('Num Examples')
plt.xlabel('Class')
plt.savefig('./Figures/TrainDataClassDistribution.png')
plt.show()

In [ ]:
lb = test_set.labels.astype(np.int)
unique, counts = np.unique(lb, return_counts=True)
plt.bar(unique, counts)
plt.xticks(np.arange(38, step = 5))
plt.title('Test Data Class Distribution')
plt.ylabel('Num Examples')
plt.xlabel('Class')
plt.savefig('./Figures/TestDataClassDistribution.png')
plt.show()

In [ ]:
train_num = training_set.samples
test_num = test_set.samples

In [ ]:
base_model=VGG16(include_top=False,input_shape=(224,224,3),weights='imagenet')
base_model.trainable=False

classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
classifier.add(Dense(38,activation='softmax'))
base_model.summary()
classifier.summary()

In [ ]:
plot_model(base_model, to_file='./figures/modelVGG.png')

In [ ]:
plot_model(classifier, to_file='./figures/firstModel.png')

In [ ]:
classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = classifier.fit_generator(generator=training_set,
                         steps_per_epoch=train_num//batch_size,
                         epochs=10,
                         validation_data=valid_set
                         )

In [ ]:
#Save history for later plotting
hist_df = pd.DataFrame(history.history)
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
classifier.save('C:/Users/frank/Documents/HomeWork/Deep Learning/Plant Disease Project/Model2')

In [ ]:
#Graph Model's Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('VGG16 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('./Figures/VGGLoss.png')
plt.show()

In [ ]:
#Graph Model's Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('VGG16 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('./Figures/VGGLoss.png')
plt.show()

In [ ]:
lClassifier = keras.models.load_model('C:/Users/frank/Documents/HomeWork/Deep Learning/Plant Disease Project/Models/')

## Evaluating the VGG16 model

In [ ]:
predTest = lClassifier.predict(test_set)

In [ ]:
#Confusion Matrix Test
predTest = np.argmax(predTest, axis=1)
predTest = predTest.tolist()

In [ ]:
matrix = confusion_matrix(test_set.labels,predTest)
matrix
matshow(matrix)

In [ ]:
df_cm = pd.DataFrame(matrix, index = labels,
                  columns = labels)
plt.figure(figsize = (25,25))

file = sn.heatmap(df_cm, annot=True)
file = file.get_figure()
file.savefig('./Figures/vgg16ConfusionMatrix2.png')

In [ ]:
print(classification_report(test_set.labels, predTest))

In [ ]:
x_testVGG = []
for i in range(1000,1005):
    x_testVGG.append(x_test[i]/255)
x_testVGG = np.array(x_testVGG)

In [ ]:
def predClassVGG(im, labels, model):
    pred = model.predict(im)
    pred = np.argmax(pred,axis=1)
    pred =pred.tolist()
    wrong = 0
    total = len(im)
    
    for i in range(len(pred)):      
        print('Predicted class: ',labelDict.get(pred[i]))
        print('Actual class: ', labelDict.get(labels[i]))
        
        fig,ax = plt.subplots(1)
        ax.imshow(im[i])
        
        if pred[i] != labels[i]:
            wrong+=1
    print("Total wrong: ", wrong)
    print('Total accuracy: ', ((total-wrong)/total)*100,'%')

In [ ]:
predClassVGG(x_testVGG,test_set.labels[1000:1005],lClassifier)